In [5]:
import glob
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [16]:
def first_iso_cleaning(sheet_name):
    file_name = "../raw_data/isotop monthly.xlsx"
    df = pd.read_excel(file_name, sheet_name=sheet_name)
    df = df.drop([df.columns[0], df.columns[1]], axis=1)
    df = df.replace("E", np.nan)
    df = df.replace(-999, np.nan)
    df = df.round(3)
    df = df.drop([60, 61, 62], axis=1)
    df = df.rename(columns={63:60, 64:61, 65:62})
    return df

In [17]:
o18 = first_iso_cleaning("d18(OBS)")
h2 = first_iso_cleaning("dD(OBS)_2")

In [18]:
pr = pd.read_excel("../raw_data/isotop monthly.xlsx", sheet_name="gsprec")
pr = pr.iloc[8:93,:] # Sep 10 - Sep 17
pr = pr.drop([pr.columns[0], pr.columns[1], pr.columns[2],
              pr.columns[3], pr.columns[4], pr.columns[5], 
              pr.columns[6]], axis=1).reset_index(drop=True)
pr = pr.drop([60, 61, 62], axis=1)
pr = pr.rename(columns={63:60, 64:61, 65:62})

In [19]:
o18.to_csv("../raw_data/raw_clean/o18_clean.csv", index=False)
h2.to_csv("../raw_data/raw_clean/h2_clean.csv", index=False)
pr.to_csv("../raw_data/raw_clean/pr_clean.csv", index=False)

In [20]:
date = pd.date_range(start='9/1/2010', end='10/1/2017', freq="M")

In [21]:
o18

,1,2,3,4,5,6,7,8,9,10,...,53,54,55,56,57,58,59,60,61,62
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,-7.323,-9.121,-12.884,-9.810,NaN,NaN,-9.849,-8.166,-7.308,...,NaN,NaN,-3.669,NaN,NaN,NaN,-3.812,NaN,NaN,NaN
3,NaN,-10.238,-7.581,-8.709,-8.460,NaN,-8.234,-9.642,-4.907,-8.585,...,NaN,NaN,-4.691,NaN,NaN,NaN,-10.155,NaN,-8.43,-5.049
4,-6.440,-6.061,-7.702,-5.303,-9.942,NaN,-6.794,-4.969,-5.882,-2.010,...,NaN,NaN,-4.765,NaN,NaN,NaN,-6.938,NaN,-3.79,-5.765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,NaN,NaN,NaN,-9.691,NaN,NaN,NaN,NaN,NaN,-3.615,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,NaN,NaN,NaN,-6.493,-8.837,-1.818,NaN,NaN,-4.215,NaN,...,-6.683,-5.653,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,-2.567,0.963,-2.633,-6.339,-6.988,0.161,-2.803,-7.162,0.099,-5.981,...,-9.539,-4.148,-5.016,-3.727,-3.942,NaN,NaN,NaN,NaN,NaN
83,-4.707,-4.119,-5.110,-8.588,-8.080,-8.238,-9.129,-9.459,-7.281,-10.648,...,-10.223,-4.593,-8.337,-6.495,-9.552,-4.672,NaN,NaN,NaN,NaN


In [22]:
def d_excess(h2, o18):
    return (h2 - 8*o18)

In [24]:
i=1
while i <= 62:
    frame = {"date":date, "precipitation":pr[i], "o18":o18[i], "h2":h2[i]}
    df_join = pd.DataFrame(frame).set_index("date")
    df_join["d_excess"] = d_excess(df_join["h2"], df_join["o18"])
    df_join = df_join.round(3)
    df_join = df_join.replace(np.nan, -999)
    df_join.to_csv("../processed_data/sta_data/sta_"+str(i)+".csv")
    i+=1

In [25]:
path = "../processed_data/sta_data/"
df = pd.concat(map(pd.read_csv, glob.glob(path + "/*.csv")))

In [26]:
df.to_csv(path+"all_sta.csv", index=False)